In [8]:

import timeit

from sklearn import preprocessing

from dataset.dataset import Dataset
from models.neural_net import NeuralNet
from models.sampled_net import SampledNet


In [9]:
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)

    return wrapped


In [10]:


xd = 2  # Input space dimensions
yd = 4  # Output dimension
num_samples = 1500  # Number of data points
layer_width = 100
epochs = 300

dataset = Dataset()
dataset.create_dataset_laplacian_of_gaussian(num_samples)
dataset.scale(preprocessing.MinMaxScaler(feature_range=(-2, 2)))
dataset.split_train_test(0.2)

### Train network with Adam

In [11]:
model_adam = NeuralNet()
model_adam.fit(dataset.X, dataset.y, [layer_width], epochs=epochs, validation_split=0.2)


Epoch 1/300
38/38 [==============================] - 1s 5ms/step - loss: 0.7939 - val_loss: 1.0335
Epoch 2/300
38/38 [==============================] - 0s 3ms/step - loss: 0.5739 - val_loss: 0.9761
Epoch 3/300
38/38 [==============================] - 0s 3ms/step - loss: 0.5502 - val_loss: 0.9477
Epoch 4/300
38/38 [==============================] - 0s 3ms/step - loss: 0.5379 - val_loss: 0.9199
Epoch 5/300
38/38 [==============================] - 0s 2ms/step - loss: 0.5260 - val_loss: 0.9008
Epoch 6/300
38/38 [==============================] - 0s 2ms/step - loss: 0.5185 - val_loss: 0.8833
Epoch 7/300
38/38 [==============================] - 0s 2ms/step - loss: 0.5098 - val_loss: 0.8643
Epoch 8/300
38/38 [==============================] - 0s 2ms/step - loss: 0.5020 - val_loss: 0.8511
Epoch 9/300
38/38 [==============================] - 0s 2ms/step - loss: 0.4978 - val_loss: 0.8419
Epoch 10/300
38/38 [==============================] - 0s 2ms/step - loss: 0.4934 - val_loss: 0.8212
Epoch 11/

### Runtime measurement of Weight Norm Preservation

In [12]:


model_base = model_adam

y_nn_train = model_base.predict(dataset.X)

model_sampled = SampledNet()
wrapped = wrapper(model_sampled.fit, dataset.X, model_base, layer2="bias_only", r=0.04,
                  project_onto_boundary=False, choose_x_2="norm")
print("Runtime: " + str(timeit.timeit(wrapped, number=5) / 5))



Layer1 sampling: 100%|██████████| 100/100 [00:01<00:00, 72.51it/s]

Runtime: 1.465421220008284


### Runtime measurement of Pair Selection with Proximity Optimization

In [13]:
model_sampled_opt = SampledNet()
wrapped = wrapper(model_sampled_opt.fit, dataset.X, model_base, layer2="bias_only", r=0.04,
                  project_onto_boundary=False, choose_x_2="norm_kdtree")
print("Runtime: " + str(timeit.timeit(wrapped, number=5) / 5))




Layer1 sampling: 100%|██████████| 100/100 [00:00<00:00, 168.28it/s]

Runtime: 0.6027322999900206
